In [11]:
import numpy as np
import os

def dB20_to_frac(d): 
    return 10**(d/20) 
    
def convert_npz_to_s2p(npz_file, output_file=None, z0=50):
    """
    Converts an NPZ file with mag/phase data into a .s2p Touchstone file.
    
    Expected keys in NPZ: 
    f11 (or f), mag11, phase11, mag12, phase12, mag21, phase21, mag22, phase22
    """
    
    # 1. Load the NPZ file
    try:
        data = np.load(npz_file)
        print(f"Successfully loaded {npz_file}")
        print(f"Available keys: {list(data.files)}")
    except Exception as e:
        print(f"Error loading NPZ file: {e}")
        return

    # 2. Extract Data
    # We assume 'f11' or just 'f' is the frequency vector for all parameters
    try:
        freq = data['f11'] if 'f11' in data else data['f']
        
        # S11
        m11 = dB20_to_frac(data['mag11'])
        p11 = data['phase11']
        
        # S21
        m21 = dB20_to_frac(data['mag21'])
        p21 = data['phase21']
        
        # S12
        m12 = dB20_to_frac(data['mag12'])
        p12 = data['phase12']
        
        # S22
        m22 = dB20_to_frac(data['mag22'])
        p22 = data['phase22']
        
    except KeyError as e:
        print(f"Missing required data key in NPZ file: {e}")
        return

    # 3. Determine Output Filename
    if output_file is None:
        output_file = os.path.splitext(npz_file)[0] + '.s2p'

    # 4. Write the Touchstone (.s2p) file
    try:
        with open(output_file, 'w') as f:
            # --- Header Information ---
            # ! indicates a comment
            f.write(f"! Created from {npz_file}\n")
            f.write("! Format: Frequency  MagS11 AngS11  MagS21 AngS21  MagS12 AngS12  MagS22 AngS22\n")
            
            # --- Option Line ---
            # # <FreqUnit> <Param> <Format> R <Impedance>
            # Hz: Frequency in Hertz
            # S: S-parameters
            # MA: Magnitude (Linear) and Angle (Degrees)
            # R 50: Reference impedance 50 Ohms
            f.write(f"# Hz S MA R {z0}\n")

            # --- Data Rows ---
            # NOTE: Standard 2-port Touchstone order is: 
            # Freq  S11  S21  S12  S22 
            # (Note the swapped order of 21 and 12 in the middle columns)
            
            for i in range(len(freq)):
                line = (
                    f"{freq[i]:.6e}\t"
                    f"{m11[i]:.6f} {p11[i]:.6f}\t"
                    f"{m21[i]:.6f} {p21[i]:.6f}\t"
                    f"{m12[i]:.6f} {p12[i]:.6f}\t"
                    f"{m22[i]:.6f} {p22[i]:.6f}\n"
                )
                f.write(line)
                
        print(f"Conversion complete! Saved to: {output_file}")
        
    except Exception as e:
        print(f"Error writing S2P file: {e}")

In [12]:

input_filename = '../../../../Downloads/HEMT_1p5V_Sparams.npz' 
convert_npz_to_s2p(input_filename)

Successfully loaded ../../../../Downloads/HEMT_1p5V_Sparams.npz
Available keys: ['f11', 'mag11', 'phase11', 'f12', 'mag12', 'phase12', 'f21', 'mag21', 'phase21', 'f22', 'mag22', 'phase22']
Conversion complete! Saved to: ../../../../Downloads/HEMT_1p5V_Sparams.s2p
